<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">TIPOLOGÍA Y CICLO DE VIDA DE LOS DATOS</p>
<p style="margin: 0; text-align:right;">Máster en Ciencia de Datos</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>




  
## 

#### Alumnos: Xavier Ricci y Guillermo Monzón

## PRÁCTICA 2:
#### CONTENIDO  

<div id='id00'/>


[1. PRESENTACIÓN DE LA PRÁCTICA Y DESCRIPCIÓN DEL *DATASET*](#id1)

[2. LIMPIEZA DE DATOS](#id2)

   [2.1 Pclass](#id21)  
   [2.2 Name](#id22)   
   [2.3 Sex](#id23)  
   [2.4 Sibsp](#id24)  
   [2.5 Parch](#id25)   
   [2.6 Age](#id26)  
   [2.7 Fare](#id27)  
   [2.8 Ticket](#id28)  
   [2.9 Cabin](#id29)   
   [2.10 Embarked](#id210)   
     
[3. ANÁLISIS DE LOS DATOS Y CONCLUSIONES](#id3)

   [3.1. Modelos de regresión](#id31)    
   [3.2. Combinación de modelos de regresión](#id32)  
   [3.3. Modelo Neuronal](#id33)  
   [3.4. Combinación de modelos neuronales](#id34)     

* * * 


<div id='id1'/> 

### [1. PRESENTACIÓN DE LA PRÁCTICA Y DESCRIPCIÓN DEL *DATASET* ](#id00)

Hemos escogido estudiar el dataset del Titanic de kaggle.com, porque se trata de un reto en el que participan personas que se dedican al análisis de datos de todo el mundo y nos queríamos participar en este reto. Se trata de una forma de medir nuestras capacidades dentro de este mundo profesional, ya que, aunque esta es una competición para principiantes, en ella participan muchos profesionales y equipos de personas. 
Nuestro reto inicial, era conseguir estar dentro de la lista del 10% con mejores resultados de los 23 mil equipos participantes. Cabe destacar que, aunque la mayoría de personas en esta lista están participando con códigos propios, también hay muchas personas que utilizan métodos tramposos, ya sea simplemente copiando los resultados finales y consiguiendo un imposible 100% de aciertos o utilizando los códigos de otros con alto rendimiento. 
Es de lógica entender que en cualquier situación natural estamos hablando de probabilidades a partir de una cantidad limitada de datos. Aunque se tuvieran miles de datos de cada uno de los pasajeros del Titanic una hora antes del accidente, sería imposible predecir todas las circunstancias que llevan a la supervivencia de cada una de las personas. Los que resbalaron, los que decidieron ceder su asiente en un bote salvavidas, los que pisaron a otra persona para ocupar su lugar… son circunstancias impredecibles que no se pueden calcular a partir de una docena de características. 
Entendemos que a base de mandar resultados se puede ir ajustando las respuestas a las correctas, aunque eso no sería la creación de un modelo de predicción, sino una especie de proceso de descubrimiento de una clave, y este no es nuestro objetivo. 
A decir verdad, nuestra sensación es que solo con los datos originales, nos parece fantástico poder superar el 80% de aciertos. 

Para llevar a cabo este estudio, los responsables de Kaggle, nos ofrecen un listado de  891 personas con 11 características y el resultado de supervivencia o no de cada uno de ellos. A partir de estos datos se nos propone predecir la supervivencia de 418 pasajeros a partir de las mismas 11 caracteristicas. 
 Para decidir los datos de interés a analizar entendemos que primero es necesario conocer los datos en profundidad y ver su estado, así, que empezamos directamente con el limpiado de los datos, para posteriormente decidir cuales son realmente útiles para nuestra predicción. 

Los atributos del dataset son los siguientes:  

- **PassengerId:** 
- **Survived:** indicador de supervivencia (0 - sobrevive / 1 - muere).
- **Pclass:** indicador de la clase del billete (1 - primera / 2 - segunda / 3 - tercera).
- **Name:** nombre del pasajero.
- **Sex:** sex (male - hombre  / female - mujer).
- **Age**: edad en años (si es menor de un año se estima de forma fraccionaria).
- **SibSp:** 
- **Parch:**
- **Ticket:** número de ticket.
- **Fare:** tarifa.
- **Cabin:** número de camarote.
- **Embarked:** puerto de embarque (C - Cherbourg / Q - Queenstown / S - Southamton).

Así pues, una vez realizado un proceso de limpieza del conjunto de datos trataremos de determinar cómo pudo afectar a las posibilidades de superviviencia del pasaje del Titanic en función de sus caracterísitcas: edad, sexo, clase, etc.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
import seaborn as sns
import pylab as plot

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, adam




In [ ]:
#dataset import
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

Unimos ambos *datasets* para tener una visión conjunta de los mismos.

In [ ]:
# Unimos ambos datasets
fullData = pd.concat([train, test], ignore_index=True, sort=False)
fullData.head()

In [ ]:
fullData.describe()

In [ ]:
fullData.info()

In [ ]:
fullData_NA = fullData.isna().sum()
train_NA = train.isna().sum()
test_NA = test.isna().sum()

pd.concat([train_NA, test_NA, fullData_NA], axis=1, sort = False, keys = ['train NA', 'test NA', 'fullData NA'])

Vemos que entre los datos hay muchos datos vacíos, especialmente en *Cabin* y en *Age*, también observamos 2 valores vacíos en *Embarked* y uno en *Fare*. Todos ellos serán tratados adecuadamente en el apartado de limpieza de datos.  

También vemos que de las 11 atributos 2 son floats, 5 son integers y 5 son objetos.

**Vemos algunas distribuciones iniciales de los datos**

In [ ]:
fullData['Survived'].value_counts(normalize = True)

Vemos que un 62% de los pasajeros murieron mientras que un 38 % se salvaron.

In [ ]:
fullData[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
sns.barplot(x='Pclass', y='Survived', data=fullData)

Por otra parte, podemos observar como un 63% de los pasajeros de PRIMERA CLASE sobrevivió al naufragio. Un 47 % para SEGUNDA CLASE y los que viajaban en TERCERA CLASE tan sólo sobrevivió un 24 %.

In [ ]:
fullData[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
sns.barplot(x='Sex', y='Survived', data=fullData)

También se observa que las mujeres sobreviven significativamente más que los hombres.  

A continuacion relacionamos algunas variables, para ver su importancia en la supervivencia

Incorporando la **edad** a este análisis.

In [ ]:
soberviven = 'soberviven'
mueren = 'mueren'
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10, 4))
mujeres = fullData[fullData['Sex']=='female']
hombres = fullData[fullData['Sex']=='male']
ax = sns.distplot(mujeres[mujeres['Survived']==1].Age.dropna(), bins=18, label = soberviven, ax = axes[0], kde =False)
ax = sns.distplot(mujeres[mujeres['Survived']==0].Age.dropna(), bins=40, label = mueren, ax = axes[0], kde =False)
ax.legend()
ax.set_title('Mujeres')
ax = sns.distplot(hombres[hombres['Survived']==1].Age.dropna(), bins=18, label = soberviven, ax = axes[1], kde = False)
ax = sns.distplot(hombres[hombres['Survived']==0].Age.dropna(), bins=40, label = mueren, ax = axes[1], kde = False)
ax.legend()
_ = ax.set_title('Hombres')

Observamos que los niños también tienen tasas de supervivencia muy superiores comparados con los adultos. 

Otra característica que podemos observar de un análisis previo de los datos es que lo pasajeros que embarcan en el puerto francés de Chebourg sobreviven de forma notoriamente superior a los pasajeros del resto de puertos

In [ ]:
fullData[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
sns.barplot(x='Embarked', y='Survived', data=fullData)

El puerto de de embarque también parece ser un factor importante a la hora de predecir la supervivencia. 

<div id='id2'/>

## [2. LIMPIEZA DE DATOS](#id00)


Para realizar la limpieza y ajuste de los datos vamos a ir tratando variable a variable del dataset original.

In [ ]:
fullData.columns

Comenzamos eliminando las variables *PassengerId* y *Survived*, pues no aportan nada para el análisis.  

<div id='id21'/> 

### [2.1 Pclass](#id00)

In [ ]:
len(fullData[fullData['Pclass'].isna()])

<div id='id22'/> 

### [2.2 Name](#id00)

In [ ]:
len(fullData[fullData['Name'].isna()])

A partir de la varible *name* podemos determinar el titutlo que nos va a ser útil más adelante. Procedemos a extraer esta información. 

In [ ]:
fullData['Title'] = fullData.Name.str.extract(r',\s*([^\.]*)\s*\.', expand=False)

In [ ]:
pd.unique(fullData['Title'])

In [ ]:
fullData['Title'].value_counts()

###### Agunas notas sobre los títulos:

Nobles: the Countess, lady is what you use to address someone of Nobility.  

Mlle = Miss

Madame = Mrs.  Usually, a servant (in Britain) addersses her Mistress as Madame.  But only if the mistress is married. 


Master = title for an underage male. If a person is under 18. En el caso del titanic todos los masters son de menos de 14.5 años. 

Colonel is a honorary title of conferred by several states in the US and certain military units of the Commonwealth of Nations.


"Ms" is a recent term for those ladies who don't think anyone needs to know whether they are married or not, like the generic 


Jonkheer is an honorific in the Low Countries denoting the lowest rank within the nobility


Veamos que relación tienen con el índice de supervivencia 

In [ ]:
def sobrevivenxtitulo(titulo):
    a=(len(fullData[(fullData['Survived']==0) & (fullData['Title']==titulo)]))
    b=(len(fullData[(fullData['Survived']==1) & (fullData['Title']==titulo)]))
    print(f" de {(a+b)}, {titulo} sobreviven: {b}, un {b/(a+b)}")

titulos = pd.unique(fullData['Title'])
for titulo in titulos:
    sobrevivenxtitulo(titulo)

De estos datos esperamos extraer la importancia de la clase y la profesion, para poderlos agrupar en conjuntos más relevantes

In [ ]:
Observamos los siguiente:

En los hombres:

Sr (Adultos en general): Con muy poco indice de supervivencia. Apenas el 15% sobreviven.

Master (Jovenes): Aunque a primera vista parecia que Master podia referirse a la clase social aparte de a la edad comprobamos a continuacion que simplemente se refiere a los menores de 15 anyos. De los jovenes el 57% sobreviven

Reverendos: Vemos que todos mueren (6 de 6 ya empieza a ser un valor interesante) 0% sobreviven

Doctores: 2 de 6 el 33% sobreviven

Major, Col y Capt (Militares) vemos que los rangos militares sobreviven en un 3/5, un 60% sobreviven

Los nobles: Sir, Don y Jonkheer 1/3 baja al 33.3% sobreviven

En las mujeres:

Mrs y Mme (Casadas): Con muy alto indice de supervivencia del 79% sobreviven

Miss y Mille (Solteras 0 y 63 años): 69% sobreviven

El resto son muy pocos casos para generalizar pero vemos que una doctora sobrevive y que la Lady y la Countess sobreviven.

In [ ]:
#dataset import
dataTrainXY = pd.read_csv("./train.csv")
dataTestX = pd.read_csv("./test.csv")
frames = [dataTrainXY,dataTestX]
DF = pd.concat(frames, ignore_index=True, sort=False)
DF.tail()

In [ ]:
titulo = DF.Name.str.extract(r',\s*([^\.]*)\s*\.', expand=False)
len(titulo)

In [ ]:
#Los valores originales:  'Mr', 'Mrs', 'Miss', 'Master', 'Don','Sir','Jonkheer','Major', 'Capt', 'Col', 'Rev', 'Dr', 'Mme', 'Ms', 'Mlle', 'Lady',    'the Countess']

# Las agrupaciones propuestas:
# Major, Col y Capt = Militares
# Sir, Don, Jonkheer, Lady y Countess  = Aristocratas
# Mrs y Mme (Casadas)
# Miss y Mlle (Solteras) tb incluyo Ms (porque entiendo que se trata de mujeres solas que querian evitar la presion social de no estar casadas)

lista = []
for word in titulo:
    if word == 'Don':
        lista.append('Arist')
    elif word =='Sir':
        lista.append('Arist')
    elif word =='Jonkheer':
        lista.append('Arist')
    
    elif word == 'Major':
        lista.append('Army')
    elif word =='Col':
        lista.append('Army')
    elif word =='Capt':
        lista.append('Army')
            
        
    elif word =='Mme':
        lista.append('Mrs')      
        
    elif word =='Mlle':
        lista.append('Miss')      
        
    elif word =='Ms':
        lista.append('Miss')  
        
    elif word == 'Lady':
        lista.append('Arist')
    elif word == 'the Countess':
        lista.append('Arist')
    elif word == 'Dona':
        lista.append('Arist')
          
    else:
        lista.append(word)
print(len(lista))
print(pd.unique(lista))


In [ ]:
DF['Title'] = lista
DF.tail()

Confirmamos la edad máxima de los *Master* y nos aseguramos que no haya niños no *Master*.

In [ ]:
max(DF[DF['Title']=="Master"].Age)

In [ ]:
DF[(DF["Age"]<15)&(DF["Title"]!="Master")&(DF["Sex"]=="male")]

Convertimos a estos niños en *Master* para unificar el criterio.

In [ ]:
DF.loc[(DF["Age"]<15)&(DF["Title"]!="Master")&(DF["Sex"]=="male") , "Title" ] = 'Master'
DF[(DF["Age"]<15)&(DF["Title"]!="Master")&(DF["Sex"]=="male")]




<div id='id23'/> 

### [2.3 Sex](#id00)

La variable *Sex* no tiene datos vacíos.

In [ ]:
len(DF[DF["Sex"].isna()])

<div id='id24'/> 

### [2.4 SibSp](#id00)
Número de hermanos / esposas-maridos a bordo. 0 si se viajaba solo.  
Definición en el dataset:  
 *Sibling = brother, sister, stepbrother, stepsister
 *Spouse = husband, wife (mistresses and fiancés were ignored)

La variable *Sibsp* no tiene datos vacíos.

In [ ]:
len(DF[DF["SibSp"].isna()])

<div id='id25'/> 

### [2.5 Parch](#id00)
Número de padres / niños. 0 si se viajaba soloo niños con nanny.

Definición en el dataset:  
 *Parent = mother, father
 *Child = daughter, son, stepdaughter, stepson
 *Some children travelled only with a nanny, therefore parch=0 for them.

La variable *Parch* no tiene datos vacíos.

In [ ]:
len(DF[DF["Parch"].isna()])

<div id='id26'/> 

### [2.6 Age](#id00)


La problemática de la variable *Age*, como hemos visto en el apartado de anterior, estriba en que tiene un número elevado de valores perdidos, 263 concretamente. A continuación procedemos a resolver esta problemática asignando un valor para cada uno de los registros desconocidos de la variable edad.

Para rellenar los datos vacios con la máxima precisión posible, los agrupamos según el titulo par ver si existe alguna relación entre este y la edad, como hemos visto que sucedia con los *Master*.

In [ ]:
def VerEdadXTitulo(titulo):
    medianaDeEdad = np.median(DF[(DF['Title']==titulo) & (DF["Age"].notnull())]['Age'])
    conEdad = len(DF[(DF['Title']==titulo) & (DF["Age"].notnull())]['Age'])
    sinEdad = len(DF[(DF['Title']==titulo) & (DF["Age"].isna())])
    
    print(f"La mediana de edad de los {conEdad} {titulo} con Age es de {medianaDeEdad}. Faltan: {sinEdad} ")
    
for titulo in (pd.unique(DF.Title)):
    VerEdadXTitulo(titulo)

Vemos que efecticamente los titulos estan relacionados con la edad. Así que utilizamos esta referencia para rellenar los datos.
Primero corregimos los que hay menos casos: *Dr* y *Master*, asignándoles la mediana.

In [ ]:
DF.loc[(DF['Title']=='Master') & (DF["Age"].isna()), "Age" ] = np.median(DF[(DF['Title']=='Master') & (DF["Age"].notnull())]['Age'])
DF[(DF['Title']=='Master') & (DF["Age"].isna())]


In [ ]:
DF.loc[(DF['Title']=='Dr') & (DF["Age"].isna()), "Age" ] = np.median(DF[(DF['Title']=='Dr') & (DF["Age"].notnull())]['Age'])
DF[(DF['Title']=='Dr') & (DF["Age"].isna())]

Para obtener las edades no registradas de pasajeros con los títulos de Mr, Mrs y Miss aplicaremos una regresion lineal, a partir de las variables *Pclass*, *Fare* y *Family*. Esta última la crearemos a partir de las variables *SibSP* y *Parch*.

Creamos la nueva variable *Family*.

In [ ]:
DF['Family'] = DF.SibSp + DF.Parch
DF.head()

Para poder hacer la regresion nos aseguramos primero de que no hay elementos vacios en las columnas que utilizaremos para el modelo.

In [ ]:
len(DF[DF["Pclass"].isna()])

In [ ]:
len(DF[DF["Family"].isna()])

In [ ]:
len(DF[DF["Fare"].isna()])


In [ ]:
DF[DF["Fare"].isna()]

Para completar este valor vacío en el precio del billete, utilizaremos la mediana del precio de pasajeros con las mismas características.

In [ ]:
medianaPrecio = np.median(DF[(DF['Title']=="Mr") &   (DF['Pclass']==3)  &  (DF['Fare'].notnull()) &  (DF['Sex']=='male') & (DF["Embarked"]=='S') & (DF["SibSp"]== 0)& (DF["Parch"]== 0)]['Fare'])

DF.loc[((DF['Title']=="Mr") &   (DF['Pclass']==3)  &  (DF['Fare'].isna()) &  (DF['Sex']=='male') & (DF["Embarked"]=='S') & (DF["SibSp"]== 0)& (DF["Parch"]== 0)) , "Fare" ] = medianaPrecio

DF[DF["Fare"].isna()]



Ahora ya podemos aplicar la regresión lineal y asignar la edad calculada a cada uno de los pasajeros correspondientes.

In [ ]:
from sklearn.linear_model import LinearRegression
#creamos un grupo segun el titulo con la edad sabida Known Age
def RegresionEdad(Titulo):
    KA = DF[(DF['Title']==Titulo) & (DF["Age"].notnull())]


    X = KA[['Pclass','Family','Fare']]
    Y = KA['Age']
    clf = LinearRegression()
    clf.fit(X, Y)
    print(clf.coef_)
    print(clf.intercept_)

    #creamos un grupo segun el titulo con la edad desconocida Unknown Age
    UA = DF[(DF['Title']==Titulo) & (DF["Age"].isna())]
    X = UA[['Pclass', 'Family','Fare']]
    Y=clf.predict(X)
    p=0
    for y in Y:
        if y < 0:
            Y[p]=0.555
        p=p+1
    print(Y)

    DF.loc[((DF['Title']==Titulo) & (DF["Age"].isna())) , "Age" ] = Y

    print(len(DF[(DF['Title']==Titulo) & (DF["Age"].isna())]))

In [ ]:
RegresionEdad("Mr")

In [ ]:
RegresionEdad("Mrs")

In [ ]:
RegresionEdad("Miss")

A continuación vamos a hacer un último ajuste sobre la variable *Title*. De forma que tengamos la correspondiente equivalencia femenina para el valor *Master*. A estos valores los clasificaremos como *Girl*

De igual modo vamos a crear una separación para los pasajeros mayores de 60. Clasificándolos como *MrSenior* para los hombres mayores de 60 años y *MrSSenior* 

In [ ]:
DF.loc[((DF["Age"]<15)&(DF["Title"]=="Miss")) , "Title" ] = 'Girl'
DF[(DF["Title"]=="Miss")]


In [ ]:
DF.loc[((DF["Age"]>60)&(DF["Title"]=="Mr")) , "Title" ] = 'MrSenior'
len(DF[(DF["Title"]=="MrSenior")])

In [ ]:
DF.loc[((DF["Age"]>60)&(DF["Title"]=="Mrs")) , "Title" ] = 'MrSSenior'
len(DF[(DF["Title"]=="MrSSenior")])

In [ ]:
plt.figure()
DF.boxplot(['Age'])

Recapitulando:

In [ ]:
def VerEdadXTitulo(titulo):
    medianaDeEdad = np.median(DF[(DF['Title']==titulo) & (DF["Age"].notnull())]['Age'])
    conEdad = len(DF[(DF['Title']==titulo) & (DF["Age"].notnull())]['Age'])
    sinEdad = len(DF[(DF['Title']==titulo) & (DF["Age"].isna())])
    
    print(f"La mediana de edad de los {conEdad} {titulo} con Age es de {medianaDeEdad}. Faltan: {sinEdad} ")
    
for titulo in (pd.unique(DF.Title)):
    VerEdadXTitulo(titulo)

<div id='id27'/> 

### [2.7 Fare](#id00)

Hemos podido observar que hay un grupo de personas que no pagan el billete y que en la variable Ticket aparece la anotación LINE. En general son hombres que viajan solos y embarcaron en el puerto de Southamton y que en su mayoria murieron (al margen de la clase en que viajaran). Se podría deducir de ello que se trata de personas con alguna relacion especial con la compañía propietaria del barco. Se opta por identficarles como Empleados en la variable Title. 

In [ ]:
DF[DF["Fare"]<1]

In [ ]:
DF.loc[(DF["Fare"]<1), "Title" ] = 'Empleados'

Adicionalmente, para no distorsionar la variable *Fare* se decide asignarles,en este caso, el valor de la media de la variable *Fare* según la clase a la que pertenezcan. 

In [ ]:
DF.loc[(DF["Fare"]<1)&(DF["Pclass"]==1), "Fare" ] = np.mean(DF[(DF["Pclass"]==1)]["Fare"])
DF.loc[(DF["Fare"]<1)&(DF["Pclass"]==2), "Fare" ] = np.mean(DF[(DF["Pclass"]==2)]["Fare"])
DF.loc[(DF["Fare"]<1)&(DF["Pclass"]==3), "Fare" ] = np.mean(DF[(DF["Pclass"]==3)]["Fare"])
#DF[DF["Title"]=="Empleados"]

In [ ]:
plt.figure()
DF.boxplot(['Fare'])

Una vez eliminado el precio ilógico de 0, vemos que hay un outlayer entre los precios altos, lo analizamos y lo justamos a los siguientes precios más elevados para evitar que se distorsione el analisis general de este dato

In [ ]:
DF[(DF["Fare"] == max(DF["Fare"]))]

In [ ]:
#The next tarife after 512 is:
DF[(DF["Fare"]== max(DF[DF["Fare"] != max(DF["Fare"])]["Fare"]))]

In [ ]:
DF.loc[(DF["Fare"] == max(DF["Fare"])), "Fare" ] =  263
max(DF["Fare"])

En un siguiente paso categorizamos los precios para facilitar la posterior creacion de modelos de prediccion.

In [ ]:
DF['precio'] = pd.qcut(DF["Fare"], 10, labels=["1","2","3","4","5","6","7","8","9","10"])

In [ ]:
plt.figure()
DF.boxplot(['Fare'])

<div id='id28'/> 

### [3.8 Ticket](#id00)

In [ ]:
len(DF[DF["Ticket"].isna()])

Al intentar analizar los Tickets en detalle vemos que se trata de un dato muy confuso que en algunos casos es numérico y en otros es alfanumérico, para unificar el criterio de forma simple, recogemos solo los numeros finales ignorando las letras iniciales. Posteriormente, si finalmente decidimos que este aspecto es relevante podría ser interesante volver a este apartado para recoger también las letras.

In [ ]:
#Puesto que la mayoria no tienen letras recuperamos solo el ultimo numero.

billetes = []
for ticket in DF["Ticket"]:
    billetes.append(ticket.split()[-1])
    
len(billetes)


In [ ]:
DF['billete'] = billetes
DF.head()

In [ ]:
DF.loc[(DF['billete']=='LINE'), "billete" ] = '99999'
DF['billete'] = DF['billete'].astype(int)
plt.figure()
DF.boxplot(['billete'])

Corregimos los valores extremos dandoles un valor cercano al siguiente valor máximo

In [ ]:
DF.loc[(DF['billete']>400000), "billete" ] = 400000
plt.figure()
DF.boxplot(['billete'])

In [ ]:
max(DF.billete)

In [ ]:
min(DF.billete)

In [ ]:
np.mean(DF.billete)

In [ ]:
# Prepare DF
x_var = 'billete'
groupby_var = 'Pclass'
df_agg = DF.loc[:, [x_var, groupby_var]].groupby(groupby_var)
vals = [DF[x_var].values.tolist() for i, DF in df_agg]

# Draw
plt.figure(figsize=(16,9), dpi= 80)
colors = [plt.cm.Spectral(i/float(len(vals)-1)) for i in range(len(vals))]
n, bins, patches = plt.hist(vals, 80, stacked=True, density=False, color=colors[:len(vals)])

# Decoration
plt.legend({group:col for group, col in zip(np.unique(DF[groupby_var]).tolist(), colors[:len(vals)])})
plt.title(f"Stacked Histogram of ${x_var}$ colored by ${groupby_var}$", fontsize=22)
plt.xlabel(x_var)
plt.ylabel("Frequency")

plt.xticks(ticks=bins[::3], labels=[round(b,1) for b in bins[::3]])
plt.xticks(rotation=45)
plt.show()

A partir de esta representacion visual, vemos que el número de billlete tiene claramente alguna relación con la clase, especialmente a partir del billete número 40000 aproximadamente.

Siguiendo esta apreciación visual categorizamos la variable *billete* en 5 grupos:

In [ ]:
  
DF['billete'] = DF['billete'].astype(int)
DF.loc[ DF['billete'] <= 40000, 'billete'] = 1
DF.loc[(DF['billete'] > 40000) & (DF['billete'] <= 104000), 'billete'] = 2
DF.loc[(DF['billete'] > 104000) & (DF['billete'] <= 170000), 'billete'] = 3
DF.loc[(DF['billete'] > 170000) & (DF['billete'] <= 252000), 'billete'] = 4
DF.loc[ DF['billete'] > 252000, 'billete'] = 5

In [ ]:
# Prepare DF
x_var = 'billete'
groupby_var = 'Pclass'
df_agg = DF.loc[:, [x_var, groupby_var]].groupby(groupby_var)
vals = [DF[x_var].values.tolist() for i, DF in df_agg]

# Draw
plt.figure(figsize=(16,9), dpi= 80)
colors = [plt.cm.Spectral(i/float(len(vals)-1)) for i in range(len(vals))]
n, bins, patches = plt.hist(vals, 5, stacked=True, density=False, color=colors[:len(vals)])

# Decoration
plt.legend({group:col for group, col in zip(np.unique(DF[groupby_var]).tolist(), colors[:len(vals)])})
plt.title(f"Stacked Histogram of ${x_var}$ colored by ${groupby_var}$", fontsize=22)
plt.xlabel(x_var)
plt.ylabel("Frequency")

#plt.xticks(ticks=bins[::3], labels=[round(b,1) for b in bins[::3]])
plt.xticks(rotation=45)
plt.show()

<div id='id29'/> 

### [2.9 Cabin](#id00)

In [ ]:
len(DF[DF["Cabin"].isna()])


Suponiendo que las letras hacen referencia a las distintas cubiertas del barco y considerando este hecho un factor primordial un la supervivencia, recogemos estos datos y rellenamos los desconocidos con la letra U de Unknown.

In [ ]:
#rellenamos los vacios con un codigo:
for l in DF['Cabin']:
    DF['Cabin'] = DF['Cabin'].fillna("U0")
len(DF[DF["Cabin"].isna()])

In [ ]:
pd.unique(DF["Cabin"])

A partir de estas letras, creamos una clasificación numérica para facilitar su analisis.

In [ ]:
planta = []
letras = []    
planta.append(DF['Cabin'])
for letra in planta[0]:
    if letra[0]== "A":
        letras.append(1)
    elif letra[0]== "B":
        letras.append(2)  
    elif letra[0]== "C":
        letras.append(3)
    elif letra[0]== "D":
        letras.append(4)
    elif letra[0]== "E":
        letras.append(5)
    elif letra[0]== "F":
        letras.append(6)
    elif letra[0]== "G":
        letras.append(7)
    else:
        letras.append(8)
        

len(letras)

In [ ]:
DF['Planta'] = letras
pd.unique(DF['Planta'])

In [ ]:
Prueba = DF[DF.Survived.notnull()]
sns.barplot(x='Planta', y='Survived', data=Prueba)

<div id='id210'/> 

### [2.10 Embarked](#id00)

La variable *Embarked* tan sólo tiene dos valores perdidos. Vamos a tratar darle un valor a estos dos registros a partir de los
de la variable *Fare*.

In [ ]:
len(DF[DF["Embarked"].isna()])

In [ ]:
DF[DF["Embarked"].isna()]

In [ ]:
DF["Embarked"].value_counts()

Puesto que no hay rasgos que nos indiquen el lugar de embarque, elegimos el puerto donde subieron la mayoria de personas

In [ ]:
DF.loc[( DF["Embarked"].isna()), "Embarked" ] = 'S'
DF[DF["Embarked"].isna()]

<div id='id3'/>

# [3. Análisis de los datos y conclusiones](#id00)

Para finalizar la limpieza y preparación de nuestro *dateset* realizamos unos últimos ajustes:

In [ ]:
DF2=DF
len(DF2)

Antes de probar modelos, para evitar el sobre entrenamiento, ahora que ya conocemos los datos seleccionados, seleccionamos los apartados que nos parecen más relevantes y eliminamos los que aportan menos informacion.


In [ ]:
DF2=DF2.drop('PassengerId', axis=1) # Esta informacion es irrelevante
DF2=DF2.drop('Name', axis=1) # La informacion mas relevante ya esta en Title
DF2=DF2.drop('SibSp', axis=1) # Esta informacion ya esta en Family
DF2=DF2.drop('Parch', axis=1) # Esta informacion ya esta en Family
DF2=DF2.drop('Ticket', axis=1) # Esta informacion ya esta en billete
DF2=DF2.drop('Cabin', axis=1) # Esta informacion ya esta en Planta
DF2=DF2.drop('Fare', axis=1) # Esta informacion ya esta en precio

DF2.head()

In [ ]:
DF2.info()

In [ ]:
DF2['precio']=DF2['precio'].astype(int)
DF2['Age']=DF2['Age'].astype(int)

Repasamos en primer lugar los datos con los que trabajamos

A continuacion conventimos los datos categoricos en numeros creando dummies cuando sea necesario

In [ ]:
Titulos = pd.get_dummies(DF2['Title'], prefix = "Titulo")
DF2 = pd.concat([DF2, Titulos], axis=1)
DF2=DF2.drop('Title', axis=1)
DF2.head()

In [ ]:
Puerto = pd.get_dummies(DF2['Embarked'], prefix = "Puerto")
DF2 = pd.concat([DF2, Puerto], axis=1)
DF2=DF2.drop('Embarked', axis=1)
DF2.head()

In [ ]:
Sexo = pd.get_dummies(DF2['Sex'], prefix = "Sexo")
DF2 = pd.concat([DF2, Sexo], axis=1)
DF2=DF2.drop('Sex', axis=1)
DF2.head()

In [ ]:
DF2.info()

Una vez finalizada la limpieza y preparación del *dataset* procedemos a separarlo para crear de nuevo los *dataset train* y *test*.

In [ ]:
Training = DF2[DF2.Survived.notnull()]
Training.head()

Entre los datos Training (con información del target) creamos dos sets uno X con todos las caracteristicas y otro Y con el target (la supervivencia o no)

In [ ]:
X=Training.loc[:, Training.columns != 'Survived']
Y=Training.loc[:, Training.columns == 'Survived']

A su vez, creamos dos *datasets* de subconjuntos de train y test. Uno para el entrenamiento de los modelos (X_train, y_train) y otro para su validacion (X_test, y_test).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.2)


También creamos un dataset con los datos limpios para hacer la predicción que se mandará a Kaggle al que llamamos XtestFinal

In [ ]:
Testing = DF2[DF2.Survived.isnull()]
XtestFinal=Testing.loc[:, Testing.columns != 'Survived']
XtestFinal.head()

#### A continuacion crearemos diversos modelos y comprobaremos su efectividad con la finanalidad de participar en la competición activa en Kaggle: 

  #### 1. Modelos de regresión.
  #### 2. Combinación de modelos regresión.
  #### 3. Modelo de redes neuronal.
  #### 4. Combinación del modelo de red neuronal.

<div id='id31'/>

### [1. Modelos de regresión](#id00)

In [ ]:
XtestFinal = Testing.loc[:, Testing.columns != 'Survived']

In [ ]:

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Escogemos las funciones con las que queremos generar modelos para predecir la supervivencia
logreg = LogisticRegression()
logreg_cv = LogisticRegressionCV()
rf = RandomForestClassifier()
gboost = GradientBoostingClassifier()

# Creamos un vector en el que recogeremos los resultados de la precision de cada metodo en funcion de un proceso de cross validation sobre los datos X_train, y_train 
resultados =[]

# Creamos un df para recoger los resultados de la prediccion final de cada uno de los metodos
submission= pd.DataFrame(data=DF[DF.Survived.isnull()].PassengerId)

models = [logreg, logreg_cv, rf, gboost]

for model in models:
    clf=model
    Crossval = cross_val_score(clf,X_train, y_train.values.ravel(), cv = 5, scoring='accuracy')
    #recogemos la precision de cada una de las 5 cross validation y también la media total.
    resultados.append(Crossval)
    resultados.append(np.mean(Crossval))
    
    # A continuacion, entrenamos el metodo una vez mas, pero ahora con todos los datos para cada uno de los metodos
    # ravel() returns contiguous flattened array
    clf.fit(X, Y.values.ravel())
    
    # A partir del metodo entrenado, se recoge una prediccion a partir de los datos sin taget para entregar, en el df submission
    submission[str(model)] = clf.predict(XtestFinal).astype(int)

# Identificamos cada columna del df con el nombre del metodo de predicción
submission.columns = ['PassengerId','logreg', 'logreg_cv', 'rf', 'gboost']
resultados
    

In [ ]:
XtestFinal

Vemos que en general los resultados son bastante inestables. Por ejemplo, vemos que los resultados del método de Random Forest Classifier oscila entre el 69 y el 94% de aciertos según donde se haya hecho el corte de la cross validation. Aún así, este metodo es el que consigue de media un mayor numero de aciertos, llegando al 80%


In [ ]:
submission

A partir de los resultados predichos generados por el metodo Random Forest Classifier creamos un csv para comprobar su grado de acierto en Kaggle

In [ ]:
submission_rf= pd.DataFrame(data=DF[DF.Survived.isnull()].PassengerId)
submission_rf['Survived'] =  submission.rf
submission_rf

In [ ]:
submission_rf.to_csv("Submision_rf.csv", index=False)

#### Esta prediccion ha conseguido un 0.79425 en Kaggle

<div id='id32'/>

### [2. Combinación de modelos de regresión](#id00)

Aprovechamos para hacer también un combinado de los resultados de los 3 métodos con los mejores resultados

In [ ]:
submission['suma_mejores']=(np.round((submission.logreg+submission.logreg_cv + submission.rf)/3)).astype(int) 
submission


In [ ]:

submission_suma_mejores= pd.DataFrame(data=DF[DF.Survived.isnull()].PassengerId)
submission_suma_mejores['Survived'] =  submission.suma_mejores
submission_suma_mejores

In [ ]:
submission_suma_mejores.to_csv("Submision_suma_mejores1.csv", index=False)

#### Esta predicción ha conseguido un 0.78947 en Kaggle.

<div id='id33'/>

### [3. Modelo de red neuronal](#id00)

A continuacion preparamos los datos para utilizar una red neuronal completamente conectada.

In [ ]:
#normalizamos por el max-min scaling para que los numeros dentro del rango [0,1]
DF2 = (DF2 - DF2.min()) / (DF2.max() - DF2.min())
DF2.head()


In [ ]:
Training = DF2[DF2.Survived.notnull()]
X=Training.loc[:, Training.columns != 'Survived']
Y=Training.loc[:, Training.columns == 'Survived']

X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.2)

Testing = DF2[DF2.Survived.isna()]
XtestFinal = Testing.loc[:, Testing.columns != 'Survived']


In [ ]:
X.shape

In [ ]:
# Creación de la red neuronal

from keras.layers import Dense, Dropout, BatchNormalization, Activation

# Modelo
model = Sequential()

# Capa de entrada
model.add(Dense(23, input_shape=(23,)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.4))

# Capas ocultas
model.add(Dense(50))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.4))

model.add(Dense(50, activation="relu"))
    
# Capa de salida con funcion sigmoide que da un numero entre 0 y 1
model.add(Dense(1, activation="sigmoid"))



In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# learning
model.fit(X_train, y_train, nb_epoch=400, batch_size=30)

In [ ]:
#Evaluamos ahora el metodo sobre todos los datos de los que conocemos la respuesta

results = model.evaluate(X, Y, batch_size=30)
results

Lo hemos hecho correr varias veces hasta conseguir un resultado superior al 81%

Seguimos entrenando el mismo metodo con los datos del test antes de mandarlo a Kaggle

In [ ]:
model.fit(X, Y, nb_epoch=400, batch_size=30)

In [ ]:
# creamos un df para la entrega a Kaggle
submission_NN= pd.DataFrame(data=DF[DF.Survived.isnull()].PassengerId)
prediccion = model.predict(XtestFinal)
print(prediccion)

In [ ]:
sobreviven = np.round(prediccion)
arr = np.array(sobreviven)
data=arr.flatten()
submission_NN['Survived']=data
submission_NN['Survived'] = submission_NN['Survived'].astype(int)
submission_NN

In [ ]:
submission_NN.to_csv("Submision_NN.csv", index=False)

#### Esta prediccion ha conseguido un   0.80382  en Kaggle 

Para tratar de mejorar este resultado se crea una función específica. Esta funcion recoge la precision sobre los datos test en el *dataframe* contador para valorar si añadiendo una capa o aumentando o reduciendo el numero de neuronas y su activacion se pueden mejorar los resultados.

In [ ]:
def NN_0(activacion, neurona, capa):
    #print('dentro')
    # Model 
    model = Sequential()
    #input layer
    model.add(Dense(23, input_shape=(23,)))
    model.add(BatchNormalization())
    model.add(Activation(activacion))
    model.add(Dropout(0.4))

    for x in range(capa):
        # hidden layers
        model.add(Dense(neurona))
        model.add(BatchNormalization())
        model.add(Activation(activacion))
        model.add(Dropout(0.4))

    
    model.add(Dense(neurona, activation="relu"))    
           
    # output layer
    model.add(Dense(1, activation="sigmoid"))
    
    # model compile for binary classification

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # learning
    model.fit(X_train, y_train, nb_epoch=400, batch_size=30)
    
    
    result = model.evaluate(X_test, y_test, batch_size=30)

    #LO ENTRENAMOS CON TODOS LOS DATOS PARA SACAR LOS RESULTADOS FINALES
    
    model.fit(X, Y, nb_epoch=400, batch_size=30)
    
    sobreviven = np.round(model.predict(XtestFinal))
    arr = np.array(sobreviven)
    data=arr.flatten()

    
    return result,data

In [ ]:
Contador2 = pd.DataFrame()

submission_NN4= pd.DataFrame(data=DF[DF.Survived.isnull()].PassengerId)

Activacion= ["relu",'sigmoid']

Neuronas = [25, 50, 100] #,'300']

Capas = [1, 2] #,4]



for activacion in Activacion:
    for neurona in Neuronas:
        for capa in Capas:
            
            print ('********************************************************************************************************************************************************************************')
            print ('********************************************************************************************************************************************************************************')
            print (str(str(activacion)+', '+ str(neurona)+', '+str(capa)))
            print ('********************************************************************************************************************************************************************************')
            print ('********************************************************************************************************************************************************************************')
         
            result,data = NN_0(activacion, neurona, capa)
            Contador2[str(str(activacion)+', '+ str(neurona)+', '+str(capa))] = result
            submission_NN4[str(str(activacion)+', '+ str(neurona)+', '+str(capa))]=data


In [ ]:
Contador2

In [ ]:
submission_NN4

In [ ]:
Contador2=Contador2.sort_values(by=1, ascending=False, axis=1)
Contador2

In [ ]:
xxxxxx

In [ ]:
#creamos un df para la entrega a Kaggle
submission_Best_NN_Original= pd.DataFrame(data=DF[DF.Survived.isnull()].PassengerId)

arr = np.array(submission_NN4['sigmoid, 100, 2'])
data=arr.flatten()

In [ ]:
submission_Best_NN_Original['Survived']= data
submission_Best_NN_Original['Survived'] = submission_Best_NN_Original['Survived'].astype(int)
submission_Best_NN_Original


In [ ]:
submission_Best_NN_Original.to_csv(("submission_Best_NN_Original2.csv"), index=False)

#### Esta prediccion ha conseguido un 0.78947 en Kaggle

<div id='id34'/>

### [3.4. Combinación del modelo de red neuronal](#id00)

Por último, se obtiene una combinación de los 6 resultados con redes neuronales.

In [ ]:
submission_NN42= pd.DataFrame(data=DF[DF.Survived.isnull()].PassengerId)

submission_NN42['Survived'] = (np.round((submission_NN4['sigmoid, 100, 2'] + submission_NN4['sigmoid, 25, 1'] + submission_NN4['sigmoid, 50, 1'] + submission_NN4['sigmoid, 50, 2'] + submission_NN4['relu, 25, 2'] + submission_NN4['sigmoid, 100, 1'])/6)).astype(int) 
submission_NN42

In [ ]:
submission_NN42.to_csv(("submission_sum_NN42.csv"), index=False)

#### Esta predicción ha conseguido un 0.78947 en Kaggle

Después de muchos otros intentos no parece que ninguna estructura en la red neuronal pueda mejorar estos resultados.

Entendemos que para seguir mejorando, deberíamos volver a modificar aspectos de la limpieza original de datos o crear nuevas combinaciones de elementos a partir de los ya existentes. 

Aun así, una predicción del 0.80382, que nos situa entre los 9% mejores de los 23000 participantes en este concurso de Kaggle nos parece bastante aceptable, de momento. 